In [142]:
import json
import numpy
import requests
from bs4 import BeautifulSoup
from espncricinfo.exceptions import MatchNotFoundError, NoScorecardError
import pandas as pd

class Match(object):
    def __init__(self, match_id):
        self.match_id = match_id
        self.match_url = f"https://site.api.espn.com/apis/site/v2/sports/cricket/8048/summary?event={match_id}"
        self.json_url = f"https://www.espncricinfo.com/matches/engine/match/{match_id}.json"
        self.json = self.get_json()
        self.json_api = self.get_json_api()
        self.description = self.get_description()
        self.full_scorecard_url = self.get_full_scorecard_url()
        self.toss = self.get_toss()
        self.home_team = self.get_home_team()
        self.away_team = self.get_away_team()
        self.batting_first = self.get_batting_first()
        self.batting_second = self.get_batting_second()
        # self.team = self.get_team()
        self.team1_batting_df = self.get_team1_batting_df()
        self.team2_batting_df = self.get_team2_batting_df()
        self.team1_bowling_df = self.get_team1_bowling_df()
        self.team2_bowling_df = self.get_team2_bowling_df()
        self.team1_dismissals_df = self.decider1()
        self.team2_dismissals_df = self.decider2()
        self.dream11_points = self.get_dream11_points()

        
    def get_json(self):
        r = requests.get(self.json_url)
        if r.status_code == 404:
            raise MatchNotFoundError
        elif 'Scorecard not yet available' in r.text:
            raise NoScorecardError
        else:
            return r.json()
    
    def get_json_api(self):
        r = requests.get(self.match_url)
        if r.status_code == 404:
            raise MatchNotFoundError
        elif 'Scorecard not yet available' in r.text:
            raise NoScorecardError
        else:
            return r.json()
        
    def get_full_scorecard_url(self):
        url  = self.description
        matchName = url.split(",")[1].split(":")[1].strip().split(" ")[:-2]
        firstHalf = '-'.join(matchName).lower().replace("-v-", "-vs-")
        matchNo = url.split(",")[1].split(":")[0].replace(" ","-").lower()
        finalurl = firstHalf + matchNo + "-" + str(self.match_id)
        return f"https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/{finalurl}/full-scorecard"
    
    def get_description(self):
        return self.json['description']
    
    def get_toss(self):
        return self.json_api['gameInfo']['venue']['fullName']

    
    def get_home_team(self):
        a = self.json_api['rosters'][0]['team']
        return {'name':a['abbreviation'], 'fullName' : a['displayName'], 'id' : a['id']}
    
    def get_away_team(self):
        a = self.json_api['rosters'][1]['team']
        return {'name':a['abbreviation'], 'fullName' : a['displayName'], 'id' : a['id']}
    
    # def get_team(self, team):
    #     team = []
    #     for i in self.json_api['rosters'][0]['roster']:
    #          team.append(i['athlete']['name'])
    #     return team
    
    def get_batting_first(self):
        a = self.json_api['matchcards'][0]
        teamName = a['teamName']
        otherTeamName = [self.home_team['name'],self.away_team['name']]
        otherTeamName.remove(teamName)
        inningsNumber = a['inningsNumber']
        headline = a['headline']
        if(headline == 'Batting'):
            if(inningsNumber == '1'):
                return teamName
            else:
                return otherTeamName[0]
        else:
            if(inningsNumber == '2'):
                return teamName
            else:
                return otherTeamName[0]

    def get_batting_second(self):
        a = self.json_api['matchcards'][0]
        teamName = a['teamName']
        otherTeamName = [self.home_team['name'],self.away_team['name']]
        otherTeamName.remove(teamName)
        inningsNumber = a['inningsNumber']
        headline = a['headline']
        if(headline == 'Batting'):
            if(inningsNumber == '1'):
                return otherTeamName[0]
            else:
                return teamName
        else:
            if(inningsNumber == '2'):
                return otherTeamName[0]
            else:
                return teamName

    def get_team1_batting_df(self):
        r = requests.get(self.full_scorecard_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        df = pd.DataFrame(columns= ['Name', "Wicket", "Runs", "Balls", "Fours", "Sixes"])
        tables = soup.find_all('table')
        brows = tables[0].findAll('tr')[1:-4]
        for brow in brows:
            x  = brow.findAll('td')
            if len(x) == 1:
                continue
            name = x[0].text
            name2 = name.replace('(c)','')
            name3 = name2.replace('†', '')
            name3 = name3.strip()
            df.loc[len(df)] = [name3, x[1].text, x[2].text, x[3].text, x[5].text, x[6].text]
        df[["Runs", "Balls", "Fours", "Sixes"]] = df[["Runs", "Balls", "Fours", "Sixes"]].astype(int)
        df["Strike Rate"] = (df["Runs"] / df["Balls"] * 100).round(2)
        return df

    def get_team2_batting_df(self):
        r = requests.get(self.full_scorecard_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        df = pd.DataFrame(columns= ['Name', "Wicket", "Runs", "Balls", "Fours", "Sixes"])
        tables = soup.find_all('table')
        brows = tables[2].findAll('tr')[1:-4]
        for brow in brows:
            x  = brow.findAll('td')
            if len(x) == 1:
                continue
            name = x[0].text
            name2 = name.replace('(c)','')
            name3 = name2.replace('†', '')
            name3 = name3.strip()
            df.loc[len(df)] = [name3, x[1].text, x[2].text, x[3].text, x[5].text, x[6].text]
        df[["Runs", "Balls", "Fours", "Sixes"]] = df[["Runs", "Balls", "Fours", "Sixes"]].astype(int)
        df["Strike Rate"] = (df["Runs"] / df["Balls"] * 100).round(2)
        return df
    
    def get_team2_bowling_df(self):
        table = pd.read_html(self.full_scorecard_url)
        df = table[1]
        df = df[~df['BOWLING'].str[0].str.isdigit()]
        df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
        df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
        df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]] = df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]].astype("int")
        return df
    
    def get_team1_bowling_df(self):
        table = pd.read_html(self.full_scorecard_url)
        df = table[3]
        df = df[~df['BOWLING'].str[0].str.isdigit()]
        df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
        df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
        df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]] = df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]].astype("int")
        return df
    
    def get_dream11_points(self):
        return None
    
    def get_team1_dismissals_df(self):
        df = pd.DataFrame(columns= ['Name', "Position", "Dismissal Type", "Bowler", "Catcher/Runout"])
        if self.home_team['name'] == self.batting_first:
            z = 0
        else:
            z = 1

        # if self.home_team['name'] == self.batting_second:
        #     a = 1
        # else:
        #     a = 0

        for i in self.json_api['rosters'][0]['roster']:
            name = i['athlete']['name']
            position = i['athlete']['position']['name']
            dismissalType = i['linescores'][z]['statistics']['categories'][0]['stats'][4]['value']
            try:
                if(len(i['linescores'][z]['statistics']['batting']['outDetails']['fielders']) > 1):
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName'] + '/' + i['linescores'][0]['statistics']['batting']['outDetails']['fielders'][1]['athlete']['displayName']
                else:
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName']
            except:
                catcher = None
            try:
                bowler = i['linescores'][z]['statistics']['batting']['outDetails']['bowler']['displayName']
            except:
                bowler = None
            df.loc[len(df)] = [name, position, dismissalType,bowler, catcher]
        return df
    
    def get_team2_dismissals_df(self):
        df = pd.DataFrame(columns= ['Name', "Position", "Dismissal Type", "Bowler", "Catcher/Runout"])
        if self.home_team['name'] == self.batting_first:
            z = 1
        else:
            z = 0

        # if self.home_team['name'] == self.batting_first:
        #     a = 1
        # else:
        #     a = 0 

        for i in self.json_api['rosters'][1]['roster']:
            name = i['athlete']['name']
            position = i['athlete']['position']['name']
            dismissalType = i['linescores'][z]['statistics']['categories'][0]['stats'][4]['value']
            try:
                if(len(i['linescores'][z]['statistics']['batting']['outDetails']['fielders']) > 1):
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName'] + '/' + i['linescores'][0]['statistics']['batting']['outDetails']['fielders'][1]['athlete']['displayName']
                else:
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName']
            except:
                catcher = None
            try:
                bowler = i['linescores'][z]['statistics']['batting']['outDetails']['bowler']['displayName']
            except:
                bowler = None
            df.loc[len(df)] = [name, position, dismissalType,bowler, catcher]
        return df
    def decider1(self):
        if self.home_team['name'] == self.batting_first:
            return self.get_team1_dismissals_df()
        else:
            return self.get_team2_dismissals_df()
        
    def decider2(self):
        if self.home_team['name'] == self.batting_first:
            return self.get_team2_dismissals_df()
        else:
            return self.get_team1_dismissals_df()

In [143]:
matchNumber = 1426296
# t = Match(1426295)
m = Match(matchNumber)


/tmp/ipykernel_4685/3185414891.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
/tmp/ipykernel_4685/3185414891.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
/tmp/ipykernel_4685/3185414891.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [144]:
match = Match(1426261)

/tmp/ipykernel_4685/3185414891.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
/tmp/ipykernel_4685/3185414891.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
/tmp/ipykernel_4685/3185414891.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [145]:
team1batting = match.team1_batting_df
team2batting = match.team2_batting_df
team1dismiss = match.team1_dismissals_df
team2dismiss = match.team2_dismissals_df
team1bowling = match.team1_bowling_df
team2bowling = match.team2_bowling_df

In [146]:
team1batting

,Name,Wicket,Runs,Balls,Fours,Sixes,Strike Rate
0,Travis Head,c Dhawan b Arshdeep Singh,21,15,4,0,140.00
1,Abhishek Sharma,c Shashank Singh b Curran,16,11,2,1,145.45
2,Aiden Markram,c †JM Sharma b Arshdeep Singh,0,2,0,0,0.00
3,Nitish Kumar Reddy,c Rabada b Arshdeep Singh,64,37,4,5,172.97
4,Rahul Tripathi,c †JM Sharma b Patel,11,14,1,0,78.57
5,Heinrich Klaasen,c Curran b Patel,9,9,1,0,100.00
6,Abdul Samad,c Patel b Arshdeep Singh,25,12,5,0,208.33
7,Shahbaz Ahmed,not out,14,7,1,1,200.00
8,Pat Cummins,b Rabada,3,4,0,0,75.00
9,Bhuvneshwar Kumar,c Bairstow b Curran,6,8,0,0,75.00


In [147]:
team1dismiss
# problem = if home team batting first den this is wrong

,Name,Position,Dismissal Type,Bowler,Catcher/Runout
0,Travis Head,Top-order batter,c,Arshdeep Singh,Shikhar Dhawan
1,Abhishek Sharma,Allrounder,c,Sam Curran,Shashank Singh
2,Aiden Markram,Opening batter,c,Arshdeep Singh,Jitesh Sharma
3,Nitish Kumar Reddy,Batting allrounder,c,Arshdeep Singh,Kagiso Rabada
4,Rahul Tripathi,Top-order batter,c,Harshal Patel,Jitesh Sharma
5,Heinrich Klaasen,Wicketkeeper batter,c,Harshal Patel,Sam Curran
6,Abdul Samad,Batter,c,Arshdeep Singh,Harshal Patel
7,Shahbaz Ahmed,Allrounder,not out,None,None
8,Pat Cummins,Bowler,0,Kagiso Rabada,None
9,Bhuvneshwar Kumar,Bowler,c,Sam Curran,Jonny Bairstow


In [148]:
team2batting

,Name,Wicket,Runs,Balls,Fours,Sixes,Strike Rate
0,Shikhar Dhawan,st †Klaasen b Kumar,14,16,2,0,87.50
1,Jonny Bairstow,b Cummins,0,3,0,0,0.00
2,Prabhsimran Singh,c Nitish Kumar Reddy b Kumar,4,6,1,0,66.67
3,Sam Curran,c Cummins b Natarajan,29,22,2,2,131.82
4,Sikandar Raza,c †Klaasen b Unadkat,28,22,2,2,127.27
5,Shashank Singh,not out,46,25,6,1,184.00
6,Jitesh Sharma,c Abhishek Sharma b Nitish Kumar Reddy,19,11,1,1,172.73
7,Ashutosh Sharma,not out,33,15,3,2,220.00


In [149]:
team2dismiss
# Problem vice versa

,Name,Position,Dismissal Type,Bowler,Catcher/Runout
0,Shikhar Dhawan,Opening batter,st,Bhuvneshwar Kumar,Heinrich Klaasen
1,Jonny Bairstow,Wicketkeeper batter,0,Pat Cummins,None
2,Prabhsimran Singh,Wicketkeeper batter,c,Bhuvneshwar Kumar,Nitish Kumar Reddy
3,Sam Curran,Allrounder,c,T Natarajan,Pat Cummins
4,Sikandar Raza,Batting allrounder,c,Jaydev Unadkat,Heinrich Klaasen
5,Shashank Singh,Batting allrounder,not out,None,None
6,Jitesh Sharma,Wicketkeeper batter,c,Nitish Kumar Reddy,Abhishek Sharma
7,Ashutosh Sharma,Batting allrounder,not out,None,None
8,Harpreet Brar,Bowler,0,None,None
9,Harshal Patel,Bowler,0,None,None


In [150]:
team1bowling

,BOWLING,Overs,Maidens,Runs,Wickets,Economy,0s,4s,6s,WD,NB
0,Bhuvneshwar Kumar,4.0,1,32,2,8.00,13,6,0,2,0
2,Pat Cummins,4.0,0,22,1,5.50,11,3,0,0,0
4,T Natarajan,4.0,0,33,1,8.25,8,5,0,0,0
6,Nitish Kumar Reddy,3.0,0,33,1,11.00,6,3,2,1,0
8,Jaydev Unadkat,4.0,0,49,1,12.25,7,0,5,3,0
10,Shahbaz Ahmed,1.0,0,10,0,10.00,1,0,1,0,0


In [151]:
team2bowling

,BOWLING,Overs,Maidens,Runs,Wickets,Economy,0s,4s,6s,WD,NB
0,Kagiso Rabada,4.0,0,32,1,8.00,13,4,1,1,0
2,Arshdeep Singh,4.0,0,29,4,7.25,12,3,1,2,0
4,Sam Curran,4.0,0,41,2,10.25,9,2,3,1,0
6,Harshal Patel,4.0,0,30,2,7.50,8,4,0,1,0
8,Harpreet Brar,4.0,0,48,0,12.00,7,5,3,0,0


In [152]:
def generate_batting_points(x, y):
    x = x.merge(y)
    x['Fantasy Points'] = x['Runs'] + x['Fours'] + x['Sixes']*2
    x['Fantasy Points'] = numpy.where(x['Runs'] >= 50, x['Fantasy Points'] + 8, x['Fantasy Points'])
    x['Fantasy Points'] = numpy.where(x['Runs'] >= 100, x['Fantasy Points'] + 8, x['Fantasy Points'])
    x['Fantasy Points'] = numpy.where((x['Runs'] == 0) & (x['Wicket'] != 'not out '), x['Fantasy Points'] - 2 , x['Fantasy Points'])
    x['Fantasy Points'] = numpy.where((x['Strike Rate'] <=70 ) & (x['Balls'] >= 10) & (x['Position'] != 'Bowler'), x['Fantasy Points'] - 2 , x['Fantasy Points'])
    x['Fantasy Points'] = numpy.where((x['Strike Rate'] <60) & (x['Balls'] >= 10) & (x['Position'] != 'Bowler'), x['Fantasy Points'] - 2 , x['Fantasy Points'])
    x['Fantasy Points'] = numpy.where((x['Strike Rate'] <50 ) & (x['Balls'] >= 10) & (x['Position'] != 'Bowler'), x['Fantasy Points'] - 2 , x['Fantasy Points'])
    x.drop(x.iloc[:, 1:11], inplace=True, axis=1)
    return x

def generate_bowling_points(y):
    y['Fantasy Points'] = y['Wickets']*25 + y['Maidens']*8 
    y['Fantasy Points'] = numpy.where(y['Wickets'] >= 4, y['Fantasy Points'] + 8, y['Fantasy Points'])
    y['Fantasy Points'] = numpy.where(y['Wickets'] >= 5, y['Fantasy Points'] + 8, y['Fantasy Points'])
    y['Fantasy Points'] = numpy.where((y['Economy'] <= 6) & (y['Overs'] >=2) , y['Fantasy Points'] +2, y['Fantasy Points'])
    y['Fantasy Points'] = numpy.where((y['Economy'] <= 5) & (y['Overs'] >=2), y['Fantasy Points'] +2, y['Fantasy Points'])
    y.drop(y.iloc[:, 1:11], inplace=True, axis=1)
    y.rename(columns = {'BOWLING':'Name'}, inplace = True)
    return y

# def generate_points(team1batting, team1dismiss, team2batting,team2dismiss, team1bowling, team2bowling):
#     team1points = pd.concat([team1batting, team1bowling]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending= False)
#     team2points = pd.concat([team2batting, team2bowling]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending= False)
#     # adding catching points
#     return pd.concat([team1points, team2points]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending= False)

team1batting = generate_batting_points(team1batting, team1dismiss)
team2batting = generate_batting_points(team2batting, team2dismiss)
team1bowling = generate_bowling_points(team1bowling)
team2bowling = generate_bowling_points(team2bowling)
# generate_points(team1batting, team1dismiss, team2batting,team2dismiss, team1bowling, team2bowling)

In [153]:
def generate_points(team1batting, team2batting, team1bowling, team2bowling, team1dismiss, team2dismiss):
    a = generate_points1(team1batting, team2dismiss, team1bowling)
    b = generate_points2(team2batting, team1dismiss, team2bowling)
    return pd.concat([a, b]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)

def generate_points1(team1batting,team2dismiss, team1bowling):
    team1points = pd.concat([team1batting, team1bowling]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)
    return fielding1(team1points, team2dismiss)

def generate_points2(team2batting,team1dismiss, team2bowling):
    team2points = pd.concat([team2batting, team2bowling]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)
    return fielding2(team2points, team1dismiss)

def fielding1(team1points,team2dismiss):
    catcher = team2dismiss.loc[team2dismiss['Dismissal Type'] == 'c']
    catcher = catcher['Catcher/Runout']
    catcher = dict(zip(*numpy.unique(catcher, return_counts=True)))
    output = pd.DataFrame(columns= ["Name", "Fantasy Points"])
    for i in catcher:
        output.loc[len(output)] = [i, catcher[i] * 8]
    st = team2dismiss.loc[team2dismiss['Dismissal Type'] == 'st']
    st = st['Catcher/Runout']
    st = dict(zip(*numpy.unique(st, return_counts=True)))
    output3 = pd.DataFrame(columns= ["Name", "Fantasy Points"])
    for i in st:
        output3.loc[len(output3)] = [i, st[i] * 12]
    runout = team2dismiss.loc[team2dismiss['Dismissal Type'] == 'run out']
    runout = runout['Catcher/Runout']
    l = {}
    for i in runout:
        if len(i.split('/')) > 1:
            l[i.split('/')[0]] = 6
            l[i.split('/')[1]] = 6
        else:
            l[i] = 12
    output2 = pd.DataFrame(columns= ["Name", "Fantasy Points"])
    for i in l:
        output2.loc[len(output2)] = [i, l[i]]
    a = pd.concat([team1points, output]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)
    b = pd.concat([a, output3]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)
    c = pd.concat([b, output2]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)
    return c

def fielding2(team2points,team1dismiss):
    catcher = team1dismiss.loc[team1dismiss['Dismissal Type'] == 'c']
    catcher = catcher['Catcher/Runout']
    catcher = dict(zip(*numpy.unique(catcher, return_counts=True)))
    output = pd.DataFrame(columns= ["Name", "Fantasy Points"])
    for i in catcher:
        output.loc[len(output)] = [i, catcher[i] * 8]
    st = team1dismiss.loc[team1dismiss['Dismissal Type'] == 'st']
    st = st['Catcher/Runout']
    st = dict(zip(*numpy.unique(st, return_counts=True)))
    output3 = pd.DataFrame(columns= ["Name", "Fantasy Points"])
    for i in st:
        output3.loc[len(output3)] = [i, st[i] * 12]
    runout = team1dismiss.loc[team1dismiss['Dismissal Type'] == 'run out']
    runout = runout['Catcher/Runout']
    l = {}
    for i in runout:
        if len(i.split('/')) > 1:
            l[i.split('/')[0]] = 6
            l[i.split('/')[1]] = 6
        else:
            l[i] = 12
    output2 = pd.DataFrame(columns= ["Name", "Fantasy Points"])
    for i in l:
        output2.loc[len(output2)] = [i, l[i]]
    a = pd.concat([team2points, output]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)
    b = pd.concat([a, output2]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)
    c = pd.concat([b, output3]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)
    return c


generate_points(team1batting, team2batting, team1bowling, team2bowling, team1dismiss, team2dismiss)

,Name,Fantasy Points
13,Nitish Kumar Reddy,119
3,Arshdeep Singh,108
17,Sam Curran,93
5,Bhuvneshwar Kumar,64
19,Shashank Singh,62
7,Harshal Patel,58
4,Ashutosh Sharma,40
10,Jitesh Sharma,38
14,Pat Cummins,38
21,Sikandar Raza,34


In [154]:
team1points = pd.concat([team1batting, team1bowling]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)
team2points = pd.concat([team2batting, team2bowling]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending = False)

In [155]:
team1points

,Name,Fantasy Points
6,Nitish Kumar Reddy,111
3,Bhuvneshwar Kumar,64
5,Jaydev Unadkat,33
0,Abdul Samad,30
7,Pat Cummins,30
11,Travis Head,25
10,T Natarajan,25
1,Abhishek Sharma,20
9,Shahbaz Ahmed,17
8,Rahul Tripathi,12


In [156]:
team2dismiss

,Name,Position,Dismissal Type,Bowler,Catcher/Runout
0,Shikhar Dhawan,Opening batter,st,Bhuvneshwar Kumar,Heinrich Klaasen
1,Jonny Bairstow,Wicketkeeper batter,0,Pat Cummins,None
2,Prabhsimran Singh,Wicketkeeper batter,c,Bhuvneshwar Kumar,Nitish Kumar Reddy
3,Sam Curran,Allrounder,c,T Natarajan,Pat Cummins
4,Sikandar Raza,Batting allrounder,c,Jaydev Unadkat,Heinrich Klaasen
5,Shashank Singh,Batting allrounder,not out,None,None
6,Jitesh Sharma,Wicketkeeper batter,c,Nitish Kumar Reddy,Abhishek Sharma
7,Ashutosh Sharma,Batting allrounder,not out,None,None
8,Harpreet Brar,Bowler,0,None,None
9,Harshal Patel,Bowler,0,None,None


In [157]:
team1points

,Name,Fantasy Points
6,Nitish Kumar Reddy,111
3,Bhuvneshwar Kumar,64
5,Jaydev Unadkat,33
0,Abdul Samad,30
7,Pat Cummins,30
11,Travis Head,25
10,T Natarajan,25
1,Abhishek Sharma,20
9,Shahbaz Ahmed,17
8,Rahul Tripathi,12


In [158]:
team1bowling

,Name,Fantasy Points
0,Bhuvneshwar Kumar,58
2,Pat Cummins,27
4,T Natarajan,25
6,Nitish Kumar Reddy,25
8,Jaydev Unadkat,25
10,Shahbaz Ahmed,0


In [159]:
# pd.merge(team1batting , team1dismiss)

team1points = pd.concat([team1batting, team1bowling]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending= False)
# team1batting.merge(team2bowling,on=['Name'], how='outer').groupby(['Fantasy Points'], as_index=False)['count'].sum()
# team1bowling.groupby(['Name'],as_index=False).agg({'Fantasy Points': 'sum'})

In [160]:
team2points = pd.concat([team2batting, team2bowling]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending= False)

In [161]:
pd.concat([team1points, team2points]).groupby(["Name"], as_index=False)["Fantasy Points"].sum().sort_values(by = "Fantasy Points", ascending= False)

,Name,Fantasy Points
13,Nitish Kumar Reddy,111
3,Arshdeep Singh,108
17,Sam Curran,85
5,Bhuvneshwar Kumar,64
19,Shashank Singh,54
7,Harshal Patel,50
4,Ashutosh Sharma,40
21,Sikandar Raza,34
9,Jaydev Unadkat,33
14,Pat Cummins,30


In [162]:
team1batting

,Name,Fantasy Points
0,Travis Head,25
1,Abhishek Sharma,20
2,Aiden Markram,-2
3,Nitish Kumar Reddy,86
4,Rahul Tripathi,12
5,Heinrich Klaasen,10
6,Abdul Samad,30
7,Shahbaz Ahmed,17
8,Pat Cummins,3
9,Bhuvneshwar Kumar,6


In [163]:
team1bowling

,Name,Fantasy Points
0,Bhuvneshwar Kumar,58
2,Pat Cummins,27
4,T Natarajan,25
6,Nitish Kumar Reddy,25
8,Jaydev Unadkat,25
10,Shahbaz Ahmed,0


In [164]:
team2batting

,Name,Fantasy Points
0,Shikhar Dhawan,16
1,Jonny Bairstow,-2
2,Prabhsimran Singh,5
3,Sam Curran,35
4,Sikandar Raza,34
5,Shashank Singh,54
6,Jitesh Sharma,22
7,Ashutosh Sharma,40


In [165]:
team2bowling

,Name,Fantasy Points
0,Kagiso Rabada,25
2,Arshdeep Singh,108
4,Sam Curran,50
6,Harshal Patel,50
8,Harpreet Brar,0
